### Sharpe Ratio Filter and Export Notebook

1. **Notebook Objective:**
	- This notebook calculates the Sharpe Ratio for a set of stocks obtained from company_tickers.json, which is approximately all NYSE stocks
    - It uses the Sharpe Ratio as a filter to exclude underperforming stocks; this filtering will be implemented in the next notebook to ensure a complete dataset is preserved for potential future use.	
    - Only stocks with a Sharpe Ratio above a certain threshold are retained
	- The filtered stock data is then exported for further analysis or reporting  

<br>

2. **Sharpe Ratio Formula:**
	- $\text{Sharpe Ratio} = \frac{\overline{R_p} - R_f}{\sigma_p}$
        - $\overline{R_p}$: Average return of the portfolio or stock
        - $R_f$: Risk-free rate
        - $\sigma_p$: Standard deviation of portfolio or stock returns

3. Improvements:
    - Need to filter tickers which are not on yahoo finance

In [1]:
import time
import os
import sys

start_time = time.time()

WORKSPACE_DIR = os.getenv('WORKSPACE_DIR')

if not os.getcwd().endswith('portfolio_py'):
    os.chdir(f'{WORKSPACE_DIR}/portfolio_py')
print(f'Current Working Directory: {os.getcwd()}')

from utils.finance_utils import calculate_sharpe_ratio
from utils.helpers import divide_chunks
from utils.config import PROGRAM_START_DATE, PROGRAM_END_DATE, N_STOCKS_TO_GET

from dotenv import load_dotenv

from datetime import datetime
import yfinance as yf
import numpy as np
import pandas as pd


Current Working Directory: /Users/blakeuribe/Desktop/portfolio_py

---------------------------------
finance_utils.py successfully loaded, updated last April. 29 2025 4:55
---------------------------------



---------------------------------
helpers.py successfully loaded, updated last Feb. 04 2025
---------------------------------


Updated on 06/05/2025 5:56


In [2]:
N_STOCKS_TO_GET

100

In [3]:
# prevent from collecting redundant data

try:
    sharpe_data = pd.read_csv('data/clean/sharpe_ratios.csv')
    collected_date = sharpe_data['Date_Collected'][0]

    collected_date = datetime.strptime(collected_date, "%Y-%m-%d").date()
    today = datetime.today().date()

    # Check if within 7 days
    if abs((today - collected_date).days) <= 7:
        user_response = input(f"Data was collected on {collected_date}. Do you still want to proceed? (yes/no): ").strip().lower()
        if user_response == 'no':
            print("Aborting: Data already collected recently.")
            sys.exit()  # This stops the script completely

except Exception as e:
    print(f"Exception occurred: {e}")


REPLACE
nyse ticker with master csv, the idea is to collect constant data once (ie. sector) and then just store it in a master df, speeding up run time

In [4]:
starting_stock_data = pd.read_csv('data/clean/master_stock_data.csv')
starting_stock_data

,cik_str,Tickers,title,Sector,Sector_Check
0,1090872,A,"AGILENT TECHNOLOGIES, INC.",Healthcare,True
1,1675149,AA,Alcoa Corp,Basic Materials,True
2,1708646,AAAU,Goldman Sachs Physical Gold ETF,NaN,True
3,2034334,AACB,Artius II Acquisition Inc.,Financial Services,True
4,1420529,AACG,ATA Creativity Global,Consumer Defensive,True
...,...,...,...,...,...
10033,1439288,ZWS,Zurn Elkay Water Solutions Corp,Industrials,True
10034,1975641,ZYBT,Zhengye Biotechnology Holding Ltd,Healthcare,True
10035,1937653,ZYME,Zymeworks Inc.,Healthcare,True
10036,846475,ZYXI,ZYNEX INC,Healthcare,True


In [5]:
load_dotenv()

print(f'Ending Program at: {PROGRAM_END_DATE}')
print(f'Starting Program at: {PROGRAM_START_DATE}')

etf_df = pd.read_csv('data/raw/etf_data_cleaned.csv') # include all etfs with NYSE stocks
etf_df= etf_df[etf_df['Have_Data'] == True] # pre filter, as the dataset is pretty old


nyse_ticker_df = pd.read_csv('data/clean/master_stock_data.csv')
nyse_ticker_df = nyse_ticker_df.drop(columns=['cik_str', 'title', 'Sector', 'Sector_Check'])

ticker_df = pd.concat([etf_df, nyse_ticker_df]).drop_duplicates().reset_index(drop=True)


tbill_data = yf.download('^IRX', start=PROGRAM_START_DATE, end=PROGRAM_END_DATE, auto_adjust=True)['Close']
tbill_data = tbill_data / 100 / 360  # Convert to daily rate

spy_sharpe = calculate_sharpe_ratio(np.array('spy'), tbill=tbill_data, start_date=PROGRAM_START_DATE, end_date=PROGRAM_END_DATE)
print(f'Spy Sharpe: {spy_sharpe}')

Ending Program at: 2025-04-15
Starting Program at: 2024-04-15


[*********************100%***********************]  1 of 1 completed
INFO:backoff_logger:Starting call to 'utils.finance_utils.fetch_data_with_backoff', this is the 1st time calling it.
[*********************100%***********************]  1 of 1 completed

Spy Sharpe: Ticker
SPY    0.337101
dtype: float64


In [6]:
etf_df

,Tickers,Have_Data
0,AAXJ,True
1,ACWI,True
2,ACWX,True
7,ADZ,True
8,AFK,True
...,...,...
1013,XSD,True
1016,YCS,True
1017,YXI,True
1018,ZROZ,True


In [7]:
nyse_ticker_df

,Tickers
0,A
1,AA
2,AAAU
3,AACB
4,AACG
...,...
10033,ZWS
10034,ZYBT
10035,ZYME
10036,ZYXI


In [8]:

# Set chunk size and number of stocks
num_in_chunks = 15

# Try not to get redudant data
try:
    tickers_not_collected = ticker_df[~ticker_df['Tickers'].isin(sharpe_data['Tickers'])]
    tickers = tickers_not_collected['Tickers'][0:N_STOCKS_TO_GET]  

except Exception as e:
    tickers = ticker_df['Tickers'][0:N_STOCKS_TO_GET]  
    print(e)

# Get tickers and divide into chunks
ticker_chunks = list(divide_chunks(tickers, num_in_chunks))


# Initialize an empty list to store results
df_list = []

# Loop through each chunk and process the stocks
for chunk in ticker_chunks:
    try:
        # Calculate Sharpe ratios for the current chunk
        sharpe_ratios_series = calculate_sharpe_ratio(
            chunk, 
            tbill=tbill_data, 
            start_date=PROGRAM_START_DATE, 
            end_date=PROGRAM_END_DATE
        )
        # Create a DataFrame for the chunk
        results_df_chunk = pd.DataFrame({
            'Tickers': sharpe_ratios_series.index, 
            'Sharpe_ratios': sharpe_ratios_series.values
        })
        df_list.append(results_df_chunk)

    except Exception as e:
        print(f'Error calculating Sharpe ratios for chunk: {chunk}\n{str(e)}')

# Concatenate all DataFrames vertically
sharpe_df = pd.concat(df_list, axis=0, ignore_index=True)


# Ensure we have a benchmark value for futre use
sharpe_df.loc[len(sharpe_df)] = ['SPY', float(spy_sharpe.values[0])]

# filtering to save data storage

# spy_sharpe = sharpe_df.loc[sharpe_df['Tickers'] == 'SPY', 'Sharpe_ratios'].values[0]
# sharpe_df = sharpe_df[sharpe_df['Sharpe_ratios'] >= spy_sharpe]

# sharpe_df = sharpe_df.drop_duplicates()

# Check percentage of NaN values
pct_of_nan = (sharpe_df.isna().sum().sum() / len(sharpe_df) * 100).round(2)
print('\n----Df Report----')
print(f'Pct of NaN values is: {pct_of_nan}%')

# Save the results to CSV

file_path = f'{WORKSPACE_DIR}/portfolio_py/data/clean/sharpe_ratios.csv'

sharpe_df['Date_Collected'] = datetime.today().date() # Add collection date
file_exists = os.path.exists(file_path) # Check file existence

# export sharpe df
# Condition based on % of NaNs
if pct_of_nan >= 25:
    user_input = input(f"The percentage of NaN values is {pct_of_nan:.2f}%. Do you still want to export the DataFrame? (yes/no): ").strip().lower()
    if user_input == 'yes':
        sharpe_df.to_csv(file_path, mode='a', index=False, header=not file_exists)
        print('Df containing Sharpe Ratios Successfully Exported')
    else:
        print('Export cancelled.')
else:
    sharpe_df.to_csv(file_path, mode='a', index=False, header=not file_exists)
    print('Nan values are below threshold. Successfully Exported')


end_time = time.time()
elapsed_time = end_time - start_time

print('\n----Time Report----')
print(f'Processing time: {elapsed_time:.2f} seconds, for {N_STOCKS_TO_GET} Tickers')

INFO:backoff_logger:Starting call to 'utils.finance_utils.fetch_data_with_backoff', this is the 1st time calling it.
[*********************100%***********************]  15 of 15 completed
ERROR:yfinance:
3 Failed downloads:
ERROR:yfinance:['AACT-WT', 'AAM-WT']: YFPricesMissingError('possibly delisted; no price data found  (1d 2024-04-15 -> 2025-04-15)')
ERROR:yfinance:['AADI']: YFPricesMissingError('possibly delisted; no price data found  (1d 2024-04-15 -> 2025-04-15) (Yahoo error = "No data found, symbol may be delisted")')
/Users/blakeuribe/Desktop/portfolio_py/utils/finance_utils.py:131: FutureWarning: The default fill_method='pad' in DataFrame.pct_change is deprecated and will be removed in a future version. Either fill in any non-leading NA values prior to calling pct_change or specify 'fill_method=None' to not fill NA values.
  daily_returns = stock_data.pct_change()
INFO:backoff_logger:Starting call to 'utils.finance_utils.fetch_data_with_backoff', this is the 1st time calling i


----Df Report----
Pct of NaN values is: 89.11%
Df containing Sharpe Ratios Successfully Exported

----Time Report----
Processing time: 15.24 seconds, for 100 Tickers


In [9]:
sharpe_df

,Tickers,Sharpe_ratios,Date_Collected
0,AAAU,NaN,2025-07-05
1,AACB,NaN,2025-07-05
2,AACG,NaN,2025-07-05
3,AACT-UN,NaN,2025-07-05
4,AACT-WT,NaN,2025-07-05
...,...,...,...
96,ADD,-1.497423,2025-07-05
97,ADGM,-1.418823,2025-07-05
98,ADGO,-245.121638,2025-07-05
99,ADIL,-0.393940,2025-07-05
